In [1]:
import pandas as pd 
#from ml_metrics import quadratic_weighted_kappa
from scipy.optimize import fmin_powell
from sklearn import linear_model
import json

config = json.load(open('settings.json'))


In [2]:
num_classes = 8

train_ohd1 = pd.read_csv(config['train_xgb'])
test_ohd1 = pd.read_csv(config['test_xgb'])
train_ohd2 = pd.read_csv(config['train_lr'])
test_ohd2 = pd.read_csv(config['test_lr'])
train_ohd3 = pd.read_csv(config['train_rf'])
test_ohd3 = pd.read_csv(config['test_rf'])
train_ohd4 = pd.read_csv(config['train_p1'])
test_ohd4 = pd.read_csv(config['test_p1'])


In [3]:
train_ohd1.head()

,BMI,EmploymentiInfoi1,EmploymentiInfoi2,EmploymentiInfoi3,EmploymentiInfoi4,EmploymentiInfoi5,EmploymentiInfoi6,FamilyiHisti1,FamilyiHisti2,FamilyiHisti3,...,xgb4,xgb5,xgb6,xgb7,xgb8,xgb9,xgb10,xgb11,xgb12,xgb13
0,0.323008,0.028,12,1,0.0,3,-1.0000,2,-1.000000,0.598039,...,0.002680,0.025474,0.060134,0.189286,0.772670,0.079762,0.074306,0.090064,0.124930,0.118302
1,0.272288,0.000,1,3,0.0,2,0.0018,2,0.188406,-1.000000,...,0.563083,0.000373,0.005129,0.002465,0.028792,0.111417,0.283070,0.974458,0.964552,0.972092
2,0.428780,0.030,9,1,0.0,2,0.0300,3,0.304348,-1.000000,...,0.037400,0.048759,0.087142,0.081928,0.722302,0.068187,0.057908,0.076683,0.111499,0.210916
3,0.352438,0.042,9,1,0.0,3,0.2000,3,0.420290,-1.000000,...,0.003677,0.003805,0.029384,0.022346,0.958757,0.017079,0.018251,0.021350,0.022474,0.030894
4,0.424046,0.027,9,1,0.0,2,0.0500,2,0.463768,-1.000000,...,0.014576,0.020609,0.067045,0.138208,0.715174,0.075568,0.070847,0.081136,0.103498,0.166830


In [5]:
['Id']+list(train_ohd2.columns[133:])

['Id',
 'lr1',
 'lr2',
 'lr3',
 'lr4',
 'lr5',
 'lr6',
 'lr7',
 'lr8',
 'lr9',
 'lr10',
 'lr11',
 'lr12',
 'lr13']

In [4]:
ftrs2 = ['Id'] + list(train_ohd2.columns[133:])
ftrs3 = ['Id'] + list(train_ohd3.columns[133:])
ftrs4 = ['Id'] + list(train_ohd4.columns[133:])

In [5]:
train_ohd = pd.merge(train_ohd1,train_ohd2[ftrs2],on='Id')
train_ohd = pd.merge(train_ohd,train_ohd3[ftrs3],on='Id')
train_ohd = pd.merge(train_ohd,train_ohd4[ftrs4],on='Id')

In [6]:
test_ohd = pd.merge(test_ohd1,test_ohd2[ftrs2],on='Id')  
test_ohd = pd.merge(test_ohd,test_ohd3[ftrs3],on='Id')  
test_ohd = pd.merge(test_ohd,test_ohd4[ftrs4],on='Id')  

In [7]:
train_ohd.head()

,BMI,EmploymentiInfoi1,EmploymentiInfoi2,EmploymentiInfoi3,EmploymentiInfoi4,EmploymentiInfoi5,EmploymentiInfoi6,FamilyiHisti1,FamilyiHisti2,FamilyiHisti3,...,rf12,rf13,pr1,pr2,pr3,pr4,pr5,pr6,pr7,pr8
0,0.323008,0.028,12,1,0.0,3,-1.0000,2,-1.000000,0.598039,...,0.214,0.304,0.032209,0.026654,0.000931,0.002207,0.014086,0.053502,0.138284,0.732126
1,0.272288,0.000,1,3,0.0,2,0.0018,2,0.188406,-1.000000,...,0.732,0.880,0.134996,0.032046,0.058754,0.731025,0.002538,0.008319,0.006655,0.025667
2,0.428780,0.030,9,1,0.0,2,0.0300,3,0.304348,-1.000000,...,0.100,0.178,0.027351,0.041042,0.002479,0.032153,0.033197,0.072257,0.092771,0.698748
3,0.352438,0.042,9,1,0.0,3,0.2000,3,0.420290,-1.000000,...,0.044,0.054,0.004975,0.006395,0.000134,0.001750,0.002099,0.014788,0.016169,0.953691
4,0.424046,0.027,9,1,0.0,2,0.0500,2,0.463768,-1.000000,...,0.142,0.234,0.026184,0.036214,0.002235,0.012981,0.014975,0.063377,0.121909,0.722125


In [8]:
train_ohd.fillna(-1, inplace=True)
test_ohd.fillna(-1, inplace=True)


In [9]:
a0 = (0.05,0.15,0.8)
a1 = (0.05,0.15,0.4,0.4)

for j in range(1,9):
    train_ohd['p%s' % (j)] = train_ohd.apply(lambda x: a1[0]*x['lr%s' % (j)] + a1[1]*x['rf%s' % (j)] + a1[2]*x['xgb%s' %(j)] + a1[3]*x['pr%s' %(j)],1)
    test_ohd['p%s' % (j)] = test_ohd.apply(lambda x: a1[0]*x['lr%s' % (j)] + a1[1]*x['rf%s' % (j)] + a1[2]*x['xgb%s' %(j)] + a1[3]*x['pr%s' %(j)],1)            

for j in range(9,14):
    train_ohd['p%s' % (j)] = train_ohd.apply(lambda x: a0[0]*x['lr%s' % (j)] + a0[1]*x['rf%s' % (j)] + a0[2]*x['xgb%s' %(j)],1)
    test_ohd['p%s' % (j)] = test_ohd.apply(lambda x: a0[0]*x['lr%s' % (j)] + a0[1]*x['rf%s' % (j)] + a0[2]*x['xgb%s' %(j)],1)
   

In [10]:
train_ohd.head()

,BMI,EmploymentiInfoi1,EmploymentiInfoi2,EmploymentiInfoi3,EmploymentiInfoi4,EmploymentiInfoi5,EmploymentiInfoi6,FamilyiHisti1,FamilyiHisti2,FamilyiHisti3,...,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13
0,0.323008,0.028,12,1,0.0,3,-1.0000,2,-1.000000,0.598039,...,0.010496,0.025739,0.073535,0.159313,0.725436,0.087814,0.084301,0.107849,0.138885,0.153205
1,0.272288,0.000,1,3,0.0,2,0.0018,2,0.188406,-1.000000,...,0.609623,0.005005,0.029787,0.015142,0.061362,0.115136,0.277902,0.902838,0.890310,0.942684
2,0.428780,0.030,9,1,0.0,2,0.0300,3,0.304348,-1.000000,...,0.035246,0.042362,0.080036,0.086946,0.707871,0.066142,0.062269,0.075915,0.114210,0.215710
3,0.352438,0.042,9,1,0.0,3,0.2000,3,0.420290,-1.000000,...,0.002792,0.004739,0.023595,0.022099,0.950583,0.018981,0.023467,0.024362,0.027563,0.042926
4,0.424046,0.027,9,1,0.0,2,0.0500,2,0.463768,-1.000000,...,0.017220,0.022435,0.064323,0.121588,0.701039,0.073084,0.068420,0.084429,0.112391,0.183543


In [12]:
print(range(1,9))

range(1, 9)


In [13]:
columns_to_drop = ['Id', 'Response', 'MedicaliHistoryi10','MedicaliHistoryi24']
features = list(train_ohd.drop(columns_to_drop,1).columns[:129]) + ['p1','p2','p3','p4','p5','p6','p7','p8','p9','p10','p11','p12','p13'] 
features = list(train_ohd.drop(columns_to_drop,1).columns[:129]) + ['xgb1','xgb2','xgb3','xgb4','xgb5','xgb6','xgb7','xgb8','xgb9','xgb10','xgb11','xgb12','xgb13'] 


In [14]:
lr = linear_model.LinearRegression()
lr.fit(train_ohd[features],train_ohd['Response'])
train_preds = lr.predict(train_ohd[features])
test_preds = lr.predict(test_ohd[features])


In [15]:
train_preds

array([7.1295591 , 3.60125615, 7.14667532, ..., 7.17140114, 6.13521945,
       5.52844225])

In [16]:
def digit0(tuple,sr):
       
    res = []
    for y in list(sr):
        if y < tuple[0]:
            res.append(1)
        elif y < tuple[1]:
            res.append(2)
        elif y < tuple[2]:
            res.append(3)
        elif y < tuple[3]:
            res.append(4)
        elif y < tuple[4]:
            res.append(5)
        elif y < tuple[5]:
            res.append(6)
        elif y < tuple[6]:
            res.append(7)
        else: res.append(8)
    return res  

In [17]:
def digit(tuple):
    '''
    Digitize train list
    '''
    res = []
    for y in list(train_preds):
        if y < tuple[0]:
            res.append(1)
        elif y < tuple[1]:
            res.append(2)
        elif y < tuple[2]:
            res.append(3)
        elif y < tuple[3]:
            res.append(4)
        elif y < tuple[4]:
            res.append(5)
        elif y < tuple[5]:
            res.append(6)
        elif y < tuple[6]:
            res.append(7)
        else: res.append(8)
    return res        


In [18]:
from sklearn.metrics import cohen_kappa_score


In [19]:
def train_offset(tuple):
    '''
    Finding offsets
    '''
    res = digit(tuple)    
    return cohen_kappa_score(train_ohd['Response'], res)

In [20]:
x0 = (1.5,2.9,3.1,4.5,5.5,6.1,7.1)    
offsets = fmin_powell(train_offset, x0, disp = True)
final_test_preds = digit0(offsets,test_preds)
preds_out = pd.DataFrame({"Id": test_ohd['Id'].values, "Response": final_test_preds})


Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 2
         Function evaluations: 184


In [21]:
train_offset

NameError: name 'res' is not defined

In [20]:
preds_out.to_csv(config['submission'], index=0)

In [34]:
train_ohd2.columns[133:136]

Index(['lr1', 'lr2', 'lr3'], dtype='object')